In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
utilities = pd.read_csv('../data/utilities.csv',index_col=0, na_values='No', dtype={'GEOID10':'str','eiaid':'str'})
ratesdict = pd.read_excel('../data/rates19/Sales_Ult_Cust_2019.xlsx',
                      sheet_name=['States','Territories'], header=2,usecols='B:E,G:H,K:M', dtype={'Utility Number':'str'}, na_values='.')
rates = pd.concat([df for df in ratesdict.values()], ignore_index=True)
metros = gpd.read_file('../data/metro/tl_2020_us_uac10.shp')
print(rates.info())
print(utilities.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Utility Number    4498 non-null   object 
 1   Utility Name      4498 non-null   object 
 2   Part              4498 non-null   object 
 3   Service Type      4498 non-null   object 
 4   State             4498 non-null   object 
 5   Ownership         4364 non-null   object 
 6   Thousand Dollars  4373 non-null   float64
 7   Megawatthours     4372 non-null   float64
 8   Count             4373 non-null   float64
dtypes: float64(3), object(6)
memory usage: 316.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Float64Index: 17423 entries, 0.0 to nan
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GEOID10  16635 non-null  object
 1   utility  15059 non-null  object
 2   eiaid    15046 non-null  object
dtypes: object(3)
memor

In [3]:
#average of top 12 largest retail providers for TX
txrates = rates[(rates['State']=='TX')&(rates['Ownership']=='Retail Power Marketer')].nlargest(n=12,columns='Count').mean(numeric_only=True).to_dict()
txrates['Utility Name'] = 'TX Average'
rates = rates[rates['Megawatthours']>0]
#will calulate again later down the road
#rates['rate'] = rates['Thousand Dollars'] / rates['Megawatthours']
#rates['usage'] = rates['Megawatthours'] / rates['Count']
rates.head()

,Utility Number,Utility Name,Part,Service Type,State,Ownership,Thousand Dollars,Megawatthours,Count
0,34,City of Abbeville - (SC),A,Bundled,SC,Municipal,4563.0,33000.0,3384.0
1,55,City of Aberdeen - (MS),A,Bundled,MS,Municipal,3930.0,34781.0,2566.0
2,59,City of Abbeville - (LA),A,Bundled,LA,Municipal,5907.5,59232.0,4580.0
3,84,A & N Electric Coop,A,Bundled,MD,Cooperative,273.5,2094.0,272.0
4,84,A & N Electric Coop,A,Bundled,VA,Cooperative,44102.0,357988.0,31427.0


In [4]:
states = []
for index, row in metros.iterrows():
    states.append(metros.loc[index,'NAME10'].split(',')[1][1:3])
metros['State'] = states
metros.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3601 entries, 0 to 3600
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   UACE10      3601 non-null   object  
 1   GEOID10     3601 non-null   object  
 2   NAME10      3601 non-null   object  
 3   NAMELSAD10  3601 non-null   object  
 4   LSAD10      3601 non-null   object  
 5   MTFCC10     3601 non-null   object  
 6   UATYP10     3601 non-null   object  
 7   FUNCSTAT10  3601 non-null   object  
 8   ALAND10     3601 non-null   int64   
 9   AWATER10    3601 non-null   int64   
 10  INTPTLAT10  3601 non-null   object  
 11  INTPTLON10  3601 non-null   object  
 12  geometry    3601 non-null   geometry
 13  State       3601 non-null   object  
dtypes: geometry(1), int64(2), object(11)
memory usage: 394.0+ KB


In [5]:
prdf = pd.DataFrame({'GEOID10':metros[metros['State']=='PR']['GEOID10'].to_list()})
prdf['utility'] = 'PR Power'  
prdf['eiaid'] = '15497'
utilities = utilities.append(prdf)

In [6]:
utilities = utilities.dropna(subset=['GEOID10'])
utilities.GEOID10 = utilities.GEOID10.apply(lambda x: x.split('.',1)[0].zfill(5))
utilities = utilities.dropna(subset=['GEOID10']).dropna(subset=['eiaid']).drop_duplicates()
utilities

,GEOID10,utility,eiaid
0.0,24310,Commonwealth Edison Co,4110
1.0,27847,"City of Escanaba, Michigan (Utility Company)",24558
2.0,18100,"City of Clintonville, Wisconsin (Utility Company)",3814
3.0,06166,Duke Energy Indiana Inc,15470
4.0,75270,Pacific Gas & Electric Co,14328
...,...,...,...
14.0,70642,PR Power,15497
15.0,28981,PR Power,15497
16.0,97561,PR Power,15497
17.0,03034,PR Power,15497


In [7]:
utilities.GEOID10.value_counts()

76339    8
55981    8
94591    8
01819    8
88504    8
        ..
82198    1
61867    1
53200    1
96350    1
78985    1
Name: GEOID10, Length: 3255, dtype: int64

In [8]:
ratemerge = rates.merge(utilities, left_on='Utility Number', right_on='eiaid', how='inner').drop(columns=['utility'])

ratemerge

,Utility Number,Utility Name,Part,Service Type,State,Ownership,Thousand Dollars,Megawatthours,Count,GEOID10,eiaid
0,34,City of Abbeville - (SC),A,Bundled,SC,Municipal,4563.0,33000.0,3384.0,00064,34
1,55,City of Aberdeen - (MS),A,Bundled,MS,Municipal,3930.0,34781.0,2566.0,00118,55
2,123,City of Adel- (GA),A,Bundled,GA,Municipal,3988.0,27905.0,2080.0,00469,123
3,176,Ajo Improvement Co,A,Bundled,AZ,Investor Owned,502.7,4717.0,799.0,00712,176
4,183,Village of Akron - (NY),A,Bundled,NY,Municipal,954.0,18267.0,1442.0,00739,183
...,...,...,...,...,...,...,...,...,...,...,...
10318,15497,Puerto Rico Electric Pwr Authority,A,Bundled,PR,State,1314886.0,6134900.0,1341424.0,70642,15497
10319,15497,Puerto Rico Electric Pwr Authority,A,Bundled,PR,State,1314886.0,6134900.0,1341424.0,28981,15497
10320,15497,Puerto Rico Electric Pwr Authority,A,Bundled,PR,State,1314886.0,6134900.0,1341424.0,97561,15497
10321,15497,Puerto Rico Electric Pwr Authority,A,Bundled,PR,State,1314886.0,6134900.0,1341424.0,03034,15497


In [9]:
ratemerge.GEOID10.value_counts()

55981    42
45505    42
01819    42
76339    42
88504    42
         ..
56872     1
96926     1
60733     1
28954     1
78985     1
Name: GEOID10, Length: 3216, dtype: int64

In [10]:
metromerge = metros.merge(ratemerge, how='left', left_on=['GEOID10', 'State'], right_on=['GEOID10', 'State'])
for index, row in metromerge[(metromerge['State']=='TX')&metromerge['Count'].isna()].iterrows():
    for key, value in txrates.items():
        metromerge.at[index, key] = value
metromerge

,UACE10,GEOID10,NAME10,NAMELSAD10,LSAD10,MTFCC10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,...,State,Utility Number,Utility Name,Part,Service Type,Ownership,Thousand Dollars,Megawatthours,Count,eiaid
0,24310,24310,"Dixon, IL","Dixon, IL Urban Cluster",76,G3500,C,S,25562730,938057,...,IL,4110,Commonwealth Edison Co,A,Bundled,Investor Owned,2343491.1,17616341.0,2520161.0,4110
1,24310,24310,"Dixon, IL","Dixon, IL Urban Cluster",76,G3500,C,S,25562730,938057,...,IL,4110,Commonwealth Edison Co,C,Delivery,Investor Owned,572201.4,9196718.0,1137735.0,4110
2,27847,27847,"Escanaba, MI","Escanaba, MI Urban Cluster",76,G3500,C,S,46643391,283456,...,MI,24558,City of Escanaba,A,Bundled,Municipal,4337.0,34913.0,6037.0,24558
3,18100,18100,"Clintonville, WI","Clintonville, WI Urban Cluster",76,G3500,C,S,5854687,502563,...,WI,3814,City of Clintonville - (WI),A,Bundled,Municipal,1941.0,17689.0,2342.0,3814
4,06166,06166,"Bedford, IN","Bedford, IN Urban Cluster",76,G3500,C,S,30413950,2314,...,IN,15470,"Duke Energy Indiana, LLC",A,Bundled,Investor Owned,1085020.7,9246749.0,733944.0,15470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5989,10675,10675,"Brooklyn, MI","Brooklyn, MI Urban Cluster",76,G3500,C,S,7677057,129296,...,MI,4254,Consumers Energy Co,A,Bundled,Investor Owned,1978868.2,12484700.0,1611320.0,4254
5990,31450,31450,"Franklin--Highland Lake, NJ","Franklin--Highland Lake, NJ Urban Cluster",76,G3500,C,S,84306618,3337407,...,NJ,9726,Jersey Central Power & Lt Co,A,Bundled,Investor Owned,1023079.7,7788851.0,842987.0,9726
5991,31450,31450,"Franklin--Highland Lake, NJ","Franklin--Highland Lake, NJ Urban Cluster",76,G3500,C,S,84306618,3337407,...,NJ,9726,Jersey Central Power & Lt Co,C,Delivery,Investor Owned,69792.9,1633991.0,163562.0,9726
5992,16171,16171,"Chester, NY","Chester, NY Urban Cluster",76,G3500,C,S,9360430,3420,...,NY,14154,Orange & Rockland Utils Inc,A,Bundled,Investor Owned,201414.8,1037927.0,141580.0,14154


In [11]:
metromerge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5994 entries, 0 to 5993
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   UACE10            5994 non-null   object  
 1   GEOID10           5994 non-null   object  
 2   NAME10            5994 non-null   object  
 3   NAMELSAD10        5994 non-null   object  
 4   LSAD10            5994 non-null   object  
 5   MTFCC10           5994 non-null   object  
 6   UATYP10           5994 non-null   object  
 7   FUNCSTAT10        5994 non-null   object  
 8   ALAND10           5994 non-null   int64   
 9   AWATER10          5994 non-null   int64   
 10  INTPTLAT10        5994 non-null   object  
 11  INTPTLON10        5994 non-null   object  
 12  geometry          5994 non-null   geometry
 13  State             5994 non-null   object  
 14  Utility Number    5585 non-null   object  
 15  Utility Name      5741 non-null   object  
 16  Part            

In [12]:
metromap = metromerge.sort_values('Count', ascending=False).drop_duplicates(subset='GEOID10', keep='first')
#[metromerge['rate'].isnull()]
#.dropna(subset=['Count'])
metromap

,UACE10,GEOID10,NAME10,NAMELSAD10,LSAD10,MTFCC10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,...,State,Utility Number,Utility Name,Part,Service Type,Ownership,Thousand Dollars,Megawatthours,Count,eiaid
3653,79606,79606,"Sarasota--Bradenton, FL","Sarasota--Bradenton, FL Urbanized Area",75,G3500,U,S,844389439,107188680,...,FL,6452,Florida Power & Light Co,A,Bundled,Investor Owned,6655476.0,60338973.0,4479356.0,6452
897,67051,67051,"Palatka, FL","Palatka, FL Urban Cluster",76,G3500,C,S,58715239,449791,...,FL,6452,Florida Power & Light Co,A,Bundled,Investor Owned,6655476.0,60338973.0,4479356.0,6452
908,50500,50500,"Live Oak, FL","Live Oak, FL Urban Cluster",76,G3500,C,S,14498690,3873,...,FL,6452,Florida Power & Light Co,A,Bundled,Investor Owned,6655476.0,60338973.0,4479356.0,6452
4025,02912,02912,"Arcadia--Southeast Arcadia, FL","Arcadia--Southeast Arcadia, FL Urban Cluster",76,G3500,C,S,26177161,15770,...,FL,6452,Florida Power & Light Co,A,Bundled,Investor Owned,6655476.0,60338973.0,4479356.0,6452
1411,56602,56602,"Miami, FL","Miami, FL Urbanized Area",75,G3500,U,S,3177138995,222863436,...,FL,6452,Florida Power & Light Co,A,Bundled,Investor Owned,6655476.0,60338973.0,4479356.0,6452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837,29116,29116,"Faribault, MN","Faribault, MN Urban Cluster",76,G3500,C,S,29469201,592057,...,MN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5909,02440,02440,"Andrews, SC","Andrews, SC Urban Cluster",76,G3500,C,S,7781113,0,...,SC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5912,07894,07894,"Bishopville, SC","Bishopville, SC Urban Cluster",76,G3500,C,S,5967255,0,...,SC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5913,11566,11566,"Burgaw, NC","Burgaw, NC Urban Cluster",76,G3500,C,S,6407466,0,...,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
center = metromerge[metromerge.GEOID10 == '30898'].geometry.centroid
area_center = [center.y, center.x]
print(area_center)

[5947    37.829925
dtype: float64, 5947   -94.703396
dtype: float64]


<ipython-input-13-40655cbfbb46>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = metromerge[metromerge.GEOID10 == '30898'].geometry.centroid


In [14]:
testmap = folium.Map(location = area_center, zoom_start = 4)

marker_cluster = MarkerCluster().add_to(testmap)

#folium.GeoJson(metromap).add_to(testmap)

# folium.Choropleth(
#     geo_data=metromap,
#     data=metromap,
#     columns=['GEOID10','ALAND10'],
#     key_on='feature.properties.GEOID10',
#     fill_color='YlOrRd',
#     fill_opacity=0.7,
#     line_opacity=0.5,
#     bins=8
#              ).add_to(testmap)

for row_i, row_val in metromap.iterrows():
    center = row_val.geometry.centroid
    loc = [center.y,center.x]
    pop = str(row_val['NAMELSAD10']) + str(" - ") + str(row_val['GEOID10']) + str(' - ') + str(row_val['Count']) 
    if row_val['Count'] > 0:
        icon = folium.Icon(icon='hand-scissors-o', prefix='fa', color='green')
    else:
        icon = folium.Icon(icon='hand-scissors-o', prefix='fa', color='red')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    marker.add_to(marker_cluster)
    
testmap.save('../data/testmap.html')

In [15]:
metromerge[(~(metromerge['State']=='PR'))&(metromerge['Count'].isna())].nlargest(n=50,columns='ALAND10').iloc[:,1:3]

,GEOID10,NAME10
2410,22811,"Dededo--Machanao--Apotgan, GU"
4336,61840,"New Bern, NC"
5819,69517,"Pinehurst--Southern Pines, NC"
2414,16750,"Christiansted, VI"
3879,69778,"Pittsfield, MA"
3313,93862,"Wenatchee, WA"
4520,30171,"Foley, AL"
3332,22204,"Danville, IL"
5713,48691,"Lebanon, TN"
210,78877,"Sanford, NC"


In [16]:
metromap.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3601 entries, 3653 to 5947
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   UACE10            3601 non-null   object  
 1   GEOID10           3601 non-null   object  
 2   NAME10            3601 non-null   object  
 3   NAMELSAD10        3601 non-null   object  
 4   LSAD10            3601 non-null   object  
 5   MTFCC10           3601 non-null   object  
 6   UATYP10           3601 non-null   object  
 7   FUNCSTAT10        3601 non-null   object  
 8   ALAND10           3601 non-null   int64   
 9   AWATER10          3601 non-null   int64   
 10  INTPTLAT10        3601 non-null   object  
 11  INTPTLON10        3601 non-null   object  
 12  geometry          3601 non-null   geometry
 13  State             3601 non-null   object  
 14  Utility Number    3192 non-null   object  
 15  Utility Name      3348 non-null   object  
 16  Part         

In [18]:
metromap.to_file('../data/ratesbymetro.geojson', driver='GeoJSON')

In [22]:
metromap.drop(columns=['geometry','LSAD10','MTFCC10','UATYP10','FUNCSTAT10']).to_csv('../data/ratesbymetro.csv', index=False)